In [2]:
import requests
import datetime
import time
import os
import csv
import pandas as pd
# from os import environ
from sqlalchemy import MetaData, create_engine, asc
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError

import psycopg2
from psycopg2 import Error

from flask import Flask, jsonify, request
from flask_cors import cross_origin, CORS
from models.models import Base

from models.models import ExpensesRaw, Filenames
# from endpointClasses.resources import Resources
#
from sqlalchemy import Column, Integer, BigInteger, String, Text, DateTime, \
    Float, Boolean, func, ForeignKeyConstraint, Index, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref
from sqlalchemy.ext.hybrid import hybrid_property, hybrid_method
from datetime import datetime, timedelta, timezone
from geoalchemy2 import Geography, Geometry


In [3]:
### Setup the application
app = Flask(__name__)

# Wrap CORS around the app so that the server does not block machine to machine
# or browser based requests
CORS(app)

In [4]:
# Engine below for Google Cloud PostgreSQL access.
engine = create_engine('postgresql+psycopg2://postgres:5413CrossFit2018@34'
                       '.70.40.80/transgov')

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

In [5]:
col_names = ["Ministry", "Position", "Name", "Type", "Category", "Date", "Amount",
         "Description", "Receipt 1", "Receipt 2", "Receipt 3"]

In [6]:
# download  csv file
# Only do this once per day
print('Downloading .....')
df = pd.read_csv(r'https://expenses.alberta.ca/DownloadData.aspx?type=csv'
                   r'&d=IsVE/OcdpNZJ5rBbvji3qw', names=col_names,
                     low_memory=False, parse_dates=['Date'])

# Saving the dataframe
print('Saving to CSV ...')
filename = 'expenses/' + str( time.strftime('%Y%m%d') ) + ".csv"
df.to_csv(filename)
print('done...')

Saving to CSV ...
done...


In [7]:
print('Reading from file...')
# filename = "/" + filename
df=pd.read_csv(filename, names=col_names, low_memory=False, skiprows=2)
df_size = len(df.index)
print('Done...', df_size)

Reading from file...
Done... 536436


In [8]:
# save file name to filenames
filetosave = Filenames(
        name=filename,
        records = df_size,
        created_at=datetime.now()
    )
try:
    session.add(filetosave)
    session.commit()
except SQLAlchemyError as e:
    session.rollback()
    print('Could not save the new filename %s', filetosave)

In [ ]:
# compare the current download to the most previous download so that we can identify
# the changes and only process those.
# 1. Check file sizes
# 2. iterate through DF1 and lookup record in DF2
# 3.   if record exists then do nothing
# 4.   is this new record or a changed one? (hard to determine....we may have to manually see if dups become a problem)

# 4.   otherwise add record to database

In [9]:
# Process the dataframe
# Note that after the very first attempt at this we need to test for whether the record already exist and then update the
# date last found
length = len(df)
ctr = 0
for index, row in df.iterrows():
    
    newdate = df._get_value(index, 'Date')
    newdate = datetime.strptime(newdate, '%m/%d/%Y')
    newAmount = df._get_value(index, 'Amount').replace('$', '').replace(',','')
    newAmount = float(newAmount)

    if len(str(df._get_value(index, 'Receipt 1'))) > 5:
        newReceipt1 = df._get_value(index, 'Receipt 1')
    else:
        newReceipt1 = ''
    if len(str(df._get_value(index, 'Receipt 2'))) > 5:
        newReceipt2 = df._get_value(index, 'Receipt 2')
    else:
        newReceipt2 = ''
    if len(str(df._get_value(index, 'Receipt 3'))) > 5:
        newReceipt3 = df._get_value(index, 'Receipt 3')
    else:
        newReceipt3 = ''  
    
    expense = ExpensesRaw(
        ministry=df._get_value(index, 'Ministry'),
        position=df._get_value(index, 'Position'),
        name=df._get_value(index, 'Name'),
        type=df._get_value(index, 'Type'),
        category=df._get_value(index, 'Category'),
        expense_date=newdate,
        amount=newAmount,
        description=df._get_value(index, 'Description'),
        receipt1=newReceipt1,
        receipt2=newReceipt2,
        receipt3=newReceipt3,
        date_last_found=datetime.now(),
        created_at=datetime.now(),
        updated_at=datetime.now(),
    )
    
#     qry = session.query(ExpensesRaw)
#     qry = qry.filter(
#                      ExpensesRaw.ministry==df._get_value(index, 'Ministry'),
#                      ExpensesRaw.position==df._get_value(index, 'Position'),
#                      ExpensesRaw.name==df._get_value(index, 'Name'),
#                      ExpensesRaw.type==df._get_value(index, 'Type'),
#                      ExpensesRaw.category==df._get_value(index, 'Category'),
#                      ExpensesRaw.expense_date==newdate,
#                      ExpensesRaw.amount==newAmount,
#                      ExpensesRaw.description==df._get_value(index, 'Description'),
#                      ExpensesRaw.receipt1==df._get_value(index, 'Receipt 1'),
#                      ExpensesRaw.receipt2==df._get_value(index, 'Receipt 2'),
#                      ExpensesRaw.receipt3==df._get_value(index, 'Receipt 3')
#                     ).first()
#     print('Query', qry)
    
    if ctr % 100 == 0:
        print('Count: %s of %s ' %(ctr, length))
    
    ctr = ctr + 1
    
    try:
        session.add(expense)
        session.commit()
    except SQLAlchemyError as e:
#         error = str(e.__dict__['orig'])
        session.rollback()
        print('Could not save the new expense %s',
              expense)
#         print(error)
   

Count: 0 of 536436 
Count: 100 of 536436 


KeyboardInterrupt: 

In [ ]:
print(df.head())